In [ ]:
#Importing all necessary modules 
import numpy as np         
import tqdm
import gensim
from gensim.models import CoherenceModel
import pandas as pd
import os
import re
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
import spacy
import nltk
from pprint import pprint

nltk.download('stopwords')
from nltk.corpus import stopwords
os.chdir('..')

In [77]:
class LDA():    #Class for LDA
    def calculate_coh(corpus, dictionary, k, a, b,lemm):#Function for optimal number of topics based on best coherence scores 

        my_model = gensim.models.LdaMulticore(corpus=corpus,
                                               id2word=dictionary,
                                               num_topics=k, 
                                               random_state=10,
                                               chunksize=100,
                                               passes=8,
                                               alpha=a,
                                               eta=b)
       
        end_model = CoherenceModel(model=my_model, texts= lemm, dictionary=dictionary, coherence='c_v')

        return end_model.get_coherence()
    
    def give_best(corp,ids,my_texts): #For finding number of topics and the best parameters for the models based on coherence score
        minimum = 5            #minimum topics              
        maximum = 11             #maximum topics 
        jump = 1
        wide = range(minimum, maximum, jump)
        
        safe = {}
        safe['Required'] = {}
          
        B = list(np.arange(0.01, 1, 0.5))#Range for values of beta
        B.append('symmetric')
        
        A = list(np.arange(0.01, 1, 0.5)) #Range for values of alpha
        A.append('symmetric')
        A.append('asymmetric')
  
        count = len(corp)
        corp_se = [gensim.utils.ClippedCorpus(corp, int(count*0.70)), 
                       corp]

        index = ['70% of Corpus', '100% of Corpus']

        solutions = {   'My_VAS': [],
                         'my_T': [],
                         'my_A': [],
                         'my_B': [],
                         'my_COHVAL': []
                        }
       
        if 2 == 2:
            pbar = tqdm.tqdm(total=(len(A)*len(B)*len(index)*len(wide)))
            for j in range(len(corp_se)):   # for different configuration  
                for z in wide: #range of topics                 
                    for x in A: #range of alpha
                        for y in B: #range ofbeta
                            ob = LDA.calculate_coh(corpus=corp_se[j], dictionary=ids, 
                                                          k=z, a=x, b=y, lemm = my_texts)                        
                            solutions['My_VAS'].append(index[j])
                            solutions['my_T'].append(z)
                            solutions['my_A'].append(x)
                            solutions['my_B'].append(y)
                            solutions['my_COHVAL'].append(ob)
                            pbar.update(1)
            final = LDA.get_optimum(pd.DataFrame(solutions))
            pbar.close()
        return final
     
    def get_optimum(frame):# To get the optimal values of number of topics,alpha,beta based on best coherence score
        return frame.loc[frame['my_COHVAL'].idxmax()]
        
    def fit(corpus, dictionary, k, a, b): # for model fit
        my_model = gensim.models.LdaMulticore(corpus=corpus,
                                               id2word=dictionary,
                                               num_topics=k, 
                                               random_state=10,
                                               chunksize=100,
                                               passes=8,
                                               alpha=a,
                                               eta=b)
        return my_model
    
    def topics(corpus, dictionary, k, a, b): #predict new instances based on best LDA model with optimal number of topics based on best coherence score
        my_model = LDA.fit(corpus, dictionary, k, a, b)
        return my_model
    
    def display(model):
        for my_id, my_top in model.print_topics(-1):
            print("Topic: {} \nWords: {}".format(my_id, my_top ))
            print("\n")
            

In [78]:
class preprocessing():   #Class for preprocessing    
    
    def shut(done,wait):
        return [[var_1 for var_1 in simple_preprocess(str(cod)) if var_1 not in wait] for cod in done]
    
    def modification(pros,bi_gram, allowings =[ 'ADJ','NOUN', 'VERB']):
        gone = []
        for var in bi_gram:
            cod = pros(" ".join(var)) 
            gone.append([o.lemma_ for o in cod if o.pos_ in allowings])
        return gone
        
    def call(dec): # For necessary preprocessing
        pages = pd.read_csv('C:/Users/fighter/Desktop/papers.csv')
       
        pages = pages.drop(columns=['year','pdf_name' ,'event_type', 
                                       'abstract','title' ,'id' ], axis=1)
        if (dec == 1):
            pages = pages.sample(1)
        else:
            pages = pages.sample(80) #Sameples 80 units
        pages['paper_text_processed'] = pages['paper_text'].map(lambda x: re.sub('[,\.!?]', '', x))
        pages['paper_text_processed'] = pages['paper_text_processed'].map(lambda x: x.lower())
        my_data = pages.paper_text_processed.values.tolist()
        done = list(preprocessing.sending(my_data))
        bis = gensim.models.Phrases(done, min_count=8, threshold=96) # higher threshold fewer phrases.
        bii = gensim.models.phrases.Phraser(bis)
        wait = stopwords.words('english')         
        no = preprocessing.shut(done,wait)
        bi_gram = preprocessing.bi_grams(no,bii)
        pros = spacy.load("en_core_web_sm", disable=[ 'ner','parser'])
        llm = preprocessing.modification(pros,bi_gram, allowings =[ 'ADJ','NOUN', 'VERB'])
        ids = corpora.Dictionary(llm) #making the dictionary
        my = llm
        corp = [ids.doc2bow(var) for var in my]

        return ids,my,corp
    
    def sending(my_data):    
        for var in my_data:
            yield(gensim.utils.simple_preprocess(str(var), deacc=True)) 
        
    def bi_grams(no,bii):        # for bigram  
        return [bii[var] for var in no]    


In [79]:
def main():   #Main function     
    ids,my_texts,corp = preprocessing.call(0)  
    untrained = LDA.fit(corp, ids, 10 , 0.1, 0.1) # Fit model which is "NOT OPTIMAL"
    LDA.display(untrained)
    values = LDA.give_best(corp,ids,my_texts)  #evaluate optimal number of topics based on best coherence scores
    print("Optimal number of topics are : ", values[1])
    ids,my_texts,corp = preprocessing.call(1) # predict new instances(sampled 1 unit) based on best LDA model with optimal number of topics based on best coherence score
    model = LDA.topics(corp, ids, values[1], values[2], values[3])      
    LDA.display(model)
    
if __name__ == "__main__":
    main()

  0%|          | 0/144 [00:00<?, ?it/s]

Topic: 0 
Words: 0.024*"neuron" + 0.012*"cell" + 0.011*"network" + 0.010*"synaptic" + 0.008*"pyloric" + 0.007*"input" + 0.007*"pattern" + 0.006*"transmitter" + 0.005*"property" + 0.005*"potential"


Topic: 1 
Words: 0.013*"algorithm" + 0.012*"use" + 0.008*"model" + 0.008*"factor" + 0.008*"theory" + 0.007*"learn" + 0.007*"graph" + 0.006*"result" + 0.006*"player" + 0.005*"game"


Topic: 2 
Words: 0.010*"use" + 0.009*"model" + 0.009*"problem" + 0.008*"parameter" + 0.008*"network" + 0.008*"function" + 0.008*"set" + 0.007*"time" + 0.006*"process" + 0.006*"result"


Topic: 3 
Words: 0.011*"algorithm" + 0.009*"set" + 0.008*"instance" + 0.008*"use" + 0.007*"function" + 0.007*"label" + 0.006*"model" + 0.006*"datum" + 0.006*"graph" + 0.005*"method"


Topic: 4 
Words: 0.010*"use" + 0.010*"model" + 0.009*"set" + 0.009*"datum" + 0.009*"density" + 0.007*"target" + 0.007*"feature" + 0.006*"saliency" + 0.006*"number" + 0.006*"give"


Topic: 5 
Words: 0.015*"image" + 0.012*"use" + 0.011*"network" + 0.0

100%|██████████| 144/144 [1:44:44<00:00, 43.64s/it] 


             My_VAS  my_T        my_A       my_B  my_COHVAL
0     70% of Corpus     5        0.01       0.01   0.279563
1     70% of Corpus     5        0.01       0.51   0.268059
2     70% of Corpus     5        0.01  symmetric   0.279872
3     70% of Corpus     5        0.51       0.01   0.279563
4     70% of Corpus     5        0.51       0.51   0.271173
..              ...   ...         ...        ...        ...
139  100% of Corpus    10   symmetric       0.51   0.328332
140  100% of Corpus    10   symmetric  symmetric   0.328706
141  100% of Corpus    10  asymmetric       0.01   0.324620
142  100% of Corpus    10  asymmetric       0.51   0.329542
143  100% of Corpus    10  asymmetric  symmetric   0.331527

[144 rows x 5 columns]
Optimal number of topics are :  7
Topic: 0 
Words: 0.003*"matrix" + 0.003*"algorithm" + 0.003*"singular" + 0.003*"value" + 0.003*"decomposition" + 0.003*"input" + 0.003*"output" + 0.003*"figure" + 0.003*"equation" + 0.003*"learn"


Topic: 1 
Words: 0.003*"